In [1]:
import geopandas as gpd
import pandas as pd
import os
from collections import Counter
from datetime import date
import openpyxl
from win32com.client import Dispatch
from wdtools import *

In [2]:
def count_lst_ele(lst, ctnm):
    qaqc_cnt = [*map(wdcnt_dict.get, lst)]
    freq = Counter(qaqc_cnt)
    df = pd.DataFrame(sorted(freq.items()))
    df.columns = ['county', ctnm]
    return df

def read_list(setid):
    with open(os.path.join(inpath, f"{setid}_mapped.pkl"), "rb") as f:
        lst = pickle.load(f)
    return lst

# clean all formats
def removeFormatting(ws):
    # ws is not the worksheet name, but the worksheet object
    for row in ws.iter_rows():
        for cell in row:
            cell.style = 'Normal'
    return ws

In [29]:
def reformat(file):
    dat = pd.read_excel(file)
    wb = openpyxl.load_workbook(file)
    ws = wb.active
    ws = removeFormatting(ws)
    wb.save(file)
    print("Removed format")
    excel = Dispatch('Excel.Application')
    wb = excel.Workbooks.Open(file)
    excel.Worksheets(1).Activate()
    excel.ActiveSheet.Columns.AutoFit()
    wb.Close(True)
    print("Autofitted columns...")

In [3]:
sets = list(map(lambda x: 'Set00'+x, ['1', '2', '3']))

In [4]:
nm2add = [0, 1420, 2143]

In [5]:
dat = []

In [6]:
for setid in sets:
    df = combine_wd_tables(setID=setid, nm_to_add=nm2add[sets.index(setid)])
    df.loc[:, 'SetID'] = setid
    dat.append(df)

In [7]:
data=pd.concat(dat)

In [8]:
data.shape

(2878, 26)

In [9]:
data.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [10]:
data.loc[data.county=='TIllamook', 'county'] = 'Tillamook'

In [11]:
data.groupby('SetID', as_index=False)['wetdet_delin_number'].nunique()

,SetID,wetdet_delin_number
0,Set001,1003
1,Set002,434
2,Set003,433


In [12]:
ndf = data.groupby('county', as_index=False)['wetdet_delin_number'].nunique()

In [13]:
ndf.rename(columns={'wetdet_delin_number': 'total'}, inplace=True)

In [16]:
qaqc = []

In [26]:
qaqcdf = pd.DataFrame(columns=['SetID', 'Counties', 'Total Count', 'Total QAQC Count', 
                               'Partial and Mythical Count', 'Unmatched Count'])

In [27]:
for setid in sets:
    print(setid)
    counties = data[data.SetID==setid]['county'].unique()
    print(counties)
    N = data[data.SetID==setid]['wetdet_delin_number'].nunique()
    print(N)
    matched = gpd.read_file(os.path.join(inpath + '\\output\\matched\\', f'matched_records_{setid}_edited.shp'))
    a = matched[matched.notes.notnull()].wdID.unique()
    qaqc.append(a)
    unmatched = pd.read_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setid}_2.csv'))
    b = unmatched.wetdet_delin_number.unique()
    qaqc.append(b)
    print(f"partial and mythical count - {len(a)}, unmatched count - {len(b)}, total QAQC count - {len(a)+len(b)}")
    qaqcdf = qaqcdf.append({'SetID': setid,
                            'Counties': ', '.join(list(counties)),
                            'Total Count': N,
                            'Total QAQC Count': len(a)+len(b), 
                            'Partial and Mythical Count': len(a), 
                            'Unmatched Count': len(b)}, ignore_index=True)

Set001
['Clatsop' 'Coos' 'Deschutes' 'Harney' 'Jackson' 'Josephine' 'Lake'
 'Marion']
1003
partial and mythical count - 264, unmatched count - 31, total QAQC count - 295
Set002
['Baker' 'Benton' 'Clackamas' 'Morrow']
434
partial and mythical count - 138, unmatched count - 19, total QAQC count - 157
Set003
['Columbia' 'Grant' 'Klamath' 'Lincoln' 'Tillamook']
433
partial and mythical count - 127, unmatched count - 18, total QAQC count - 145


In [30]:
file=os.path.join(inpath, 'reporting', f'SetStatusReport{str(date.today()).replace("-", "")}.xlsx')

In [31]:
qaqcdf.to_excel(file, index=False)

In [32]:
reformat(file)

Removed format
Autofitted columns...


In [41]:
qaqc = unique(np.concatenate(qaqc))

In [42]:
len(qaqc)

567

In [43]:
df = data[['wetdet_delin_number', 'county']]

In [33]:
wdcnt_dict = df.set_index('wetdet_delin_number').to_dict(orient='dict')['county']

In [69]:
qaqc_df = count_lst_ele(qaqc, 'QAQC')

In [58]:
mapped = []

In [60]:
for setid in ['Set001', 'Set002']:
    mapped.append(read_list(setid))

In [61]:
mapped = unique(np.concatenate(mapped))

In [62]:
mapped

['WD2017-0347',
 'WD2018-0124',
 'WD2018-0217',
 'WD2018-0306',
 'WD2018-0386',
 'WD2018-0419',
 'WD2018-0522',
 'WD2018-0657',
 'WD2019-0124',
 'WD2019-0231',
 'WD2019-0338',
 'WD2019-0431',
 'WD2019-0544',
 'WD2020-0067',
 'WD2020-0391',
 'WD2020-0536',
 'WD2020-0668',
 'WD2020-0677',
 'WD2021-0162',
 'WD2021-0252',
 'WD2021-0559',
 'WD2021-0616',
 'WD2021-0703']

In [107]:
colnm='QAQC_comp'

In [108]:
mapped_df = count_lst_ele(mapped, colnm)

In [142]:
out = ndf.merge(qaqc_df, on='county', how='left')

In [143]:
out1 = out.merge(mapped_df, on='county', how='left')

In [145]:
sel = out1.county.isin(['Columbia', 'Grant', 'Klamath', 'Lincoln', 'Tillamook'])

In [146]:
out1.loc[sel, colnm] = out1.loc[sel, 'QAQC']

In [147]:
sel1 = out1.QAQC_comp.isnull()

In [148]:
out1.loc[sel1, colnm] = 0

In [149]:
out1

,county,total,QAQC,QAQC_comp
0,Baker,38,12,0.0
1,Benton,91,30,1.0
2,Clackamas,275,96,3.0
3,Clatsop,137,41,0.0
4,Columbia,93,33,33.0
5,Coos,120,27,3.0
6,Deschutes,172,39,5.0
7,Grant,23,6,6.0
8,Harney,13,11,2.0
9,Jackson,221,51,4.0


In [114]:
out1[colnm] = out1[colnm].apply(np.int64)

In [115]:
out1['QAQC_pct'] = out1.apply(lambda row: str(round(row.QAQC/row.total*100,1))+('%'), axis=1)

In [117]:
out1['QAQC_comp_pct'] = out1.apply(lambda row: str(round(row.QAQC_comp/row.QAQC*100,1))+('%'), axis=1)

In [118]:
out1

,county,total,QAQC,QAQC_comp,QAQC_pct,QAQC_comp_pct
0,Baker,38,12,0,31.6%,0.0%
1,Benton,91,30,1,33.0%,3.3%
2,Clackamas,275,96,3,34.9%,3.1%
3,Clatsop,137,41,0,29.9%,0.0%
4,Columbia,93,33,33,35.5%,100.0%
5,Coos,120,27,3,22.5%,11.1%
6,Deschutes,172,39,5,22.7%,12.8%
7,Grant,23,6,6,26.1%,100.0%
8,Harney,13,11,2,84.6%,18.2%
9,Jackson,221,51,4,23.1%,7.8%


In [133]:
out.total.sum()

1872

In [134]:
out.QAQC.sum()

567

In [136]:
567/1872

0.30288461538461536

In [120]:
out1.columns = ['County', 'Total Count', 'QAQC Count', 'QAQC Completed Count', 
                'QAQC Count Percentage', 'QAQC Completed Count Percentage']

In [135]:
out1['QAQC Completed Count'].sum()

161

In [137]:
161/567

0.2839506172839506

In [138]:
1872/9000

0.208

In [126]:
inpath

'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping'

In [129]:
file=os.path.join(inpath, 'reporting', f'StatusReport{str(date.today()).replace("-", "")}.xlsx')

In [127]:
out1.to_excel(file, index=False)

In [132]:
reformat(file)

Removed format
Autofitted columns...
